In [43]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime

In [2]:
# Load the data
dateparse = lambda x: datetime.strptime(x, '%Y/%m/%d') if isinstance(x, str) else x
data = pd.read_excel('Signed Players Dataset VALORANT VCT DATASET.xlsx', parse_dates=['date'], date_parser=dateparse)
test_data = pd.read_excel('Test Dataset VALORANT VCT DATASET.xlsx', parse_dates=['date'], date_parser=dateparse).dropna(how='any',axis=0)

In [3]:
# Select features and target variable
inputs = ['date', 'player']
features = ['kills_per_round', 'assists_per_round',
            'first_kills_per_round', 'first_deaths_per_round', 'headshot_percentage',
            'clutch_success_percentage', 'total_kills', 'total_deaths', 'total_assists',
            'total_first_kills', 'total_first_deaths']
target_variable = 'average_combat_score'
#Note: create models w/ target variable for rounds and rating

In [4]:
# Split variables
X_train = data[features]
X_test = test_data[features]
y_train = data[target_variable]
y_test = test_data[target_variable]

In [5]:
data = data.replace('\n','', regex=True)
data = data.replace(' ','', regex=True)
test_data = test_data.replace('\n','', regex=True)
test_data = test_data.replace(' ','', regex=True)

In [6]:
dates = data['date'].unique()

In [8]:
data['date'].unique()

array(['2022-10-07T00:00:00.000000000', '2022-08-31T00:00:00.000000000',
       '2023-02-13T00:00:00.000000000', '2021-09-19T00:00:00.000000000',
       '2023-11-06T00:00:00.000000000', '2023-06-08T00:00:00.000000000',
       '2021-12-12T00:00:00.000000000', '2021-05-30T00:00:00.000000000',
       '2022-10-04T00:00:00.000000000'], dtype='datetime64[ns]')

In [14]:
test_data['date'].unique()

array(['2021-05-30T00:00:00.000000000', '2021-09-19T00:00:00.000000000',
       '2021-12-12T00:00:00.000000000', '2022-10-04T00:00:00.000000000',
       '2022-10-07T00:00:00.000000000', '2022-08-31T00:00:00.000000000',
       '2023-02-13T00:00:00.000000000', '2023-11-06T00:00:00.000000000',
       '2023-06-08T00:00:00.000000000'], dtype='datetime64[ns]')

In [7]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

def months_from_2021(dates):
    months_from_Jan_2021 = []
    for date in dates:
        months_from_Jan_2021.append(diff_month(date, datetime.strptime('2021/01/01', '%Y/%m/%d')))
    return months_from_Jan_2021

In [8]:
data = data.assign(months_from_Jan_2021 = months_from_2021(data['date']))

In [9]:
players = data['player'].unique()

In [34]:
player_data = data.loc[data['player'] == 'Cryocells']

In [35]:
player_data['months_from_Jan_2021']

0    21
1    19
2    25
Name: months_from_Jan_2021, dtype: int64

In [28]:
player_data['kills_per_round']

0    0.89
1    0.85
2    0.75
Name: kills_per_round, dtype: float64

In [76]:
player_data['kills_per_round'].values

array([0.89, 0.85, 0.75])

In [36]:
test_model = XGBRegressor().fit(player_data['months_from_Jan_2021'].values[:,np.newaxis], player_data['kills_per_round'].values)

In [10]:
#Create regression model for each feature for each players
player_feature_models = {}
for player in players:
    player_feature_model = {}
    player_data = data.loc[data['player'] == player]
    for feature in features:
        feature_model = XGBRegressor().fit(player_data['months_from_Jan_2021'].values[:,np.newaxis], player_data[feature])
        player_feature_model.update({feature:feature_model})
    player_feature_models.update({player:player_feature_model})

In [44]:
#Save Models

filename = 'player_feature_models.sav'
pickle.dump(player_feature_models, open(filename, 'wb'))

In [11]:
player_feature_models.get('Cryocells').get('kills_per_round')

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [18]:
#Load models

filename = 'player_feature_models.sav'
filename1 = 'model.sav'
filename2 = 'model w tuning.sav'

model = pickle.load(open(filename1, 'rb'))
opt = pickle.load(open(filename2, 'rb'))
player_feature_models = pickle.load(filename, 'rb')

In [41]:
#Player predict values get
def predict_players_values(players_and_dates):
    player_predicted_values = {}
    for feature in features:
        player_values = []
        for player_and_date in players_and_dates:
            date = [dateparse(player_and_date[1])]
            date_from_Jan_2021 = months_from_2021(date)
            player_values.append(player_feature_models.get(player_and_date[0]).get(feature).predict(np.array([date_from_Jan_2021]))[0])
        player_predicted_values.update({feature: player_values})
    return pd.DataFrame(player_predicted_values)

In [71]:
player_predict_values_pd = pd.DataFrame(player_predict_values)

In [58]:
#Predict ACS
def predictACS(players_and_dates):
    return opt.predict(predict_players_values(players_and_dates))

In [62]:
predictACS([['Cryocells', '2024/1/23']])

array([216.1792], dtype=float32)

In [66]:
#Get ACS of players at next tourney date
next_tourney_date = '2024/1/23'
player_date = []
for player in players:
    player_date.append([player, next_tourney_date])
predictions = predictACS(player_date)

In [67]:
predictions

array([216.1792  , 209.28769 , 193.39705 , 221.84291 , 236.8729  ,
       171.35114 , 212.22661 , 153.8106  , 172.97282 , 255.79715 ,
       157.89622 , 141.90395 , 256.53033 , 238.19562 , 192.07802 ,
       240.43393 , 218.96324 , 137.93272 , 199.37138 , 171.31079 ,
       162.63625 , 169.20831 , 202.96542 , 230.54277 , 164.556   ,
       210.21294 , 170.71477 , 216.59006 , 195.77443 , 224.34409 ,
       136.47395 , 191.80576 , 176.01543 , 198.21962 , 195.44191 ,
       201.40059 , 225.00537 , 173.35925 , 162.51868 , 178.01234 ,
       200.35057 , 191.07172 , 238.62654 , 171.48116 , 236.55756 ,
       234.65024 , 205.5705  , 239.55046 , 183.89159 , 175.73378 ,
       205.40749 , 249.87735 , 187.92136 , 215.65076 , 230.4193  ,
       140.44354 , 154.82271 , 266.50122 , 188.34825 , 179.7715  ,
       182.96422 , 268.56815 , 159.71199 , 121.35794 , 123.30832 ,
       162.89153 , 166.68619 , 198.10059 , 197.98956 , 212.97049 ,
       147.63109 , 212.11493 , 179.05429 , 214.49107 , 196.285

In [68]:
players_predictions = {'Players':players, 'Predictions':predictions}
players_predictions_pd = pd.DataFrame(players_predictions)

In [69]:
players_predictions_pd

,Players,Predictions
0,Cryocells,216.179199
1,bang,209.287689
2,Asuna,193.397049
3,Boostio,221.842911
4,Xeppaa,236.872894
...,...,...
119,Laz,167.712814
120,SugarZ3ro,129.060822
121,sScary,201.221817
122,yay,210.622391


In [74]:
players_predictions_pd.sort_values('Predictions', ascending=False)

,Players,Predictions
61,trexx,268.568146
57,Nivera,266.501221
116,JessieVash,259.071716
12,keznit,256.530334
9,mwzera,255.797150
...,...,...
30,zekken,136.473953
120,SugarZ3ro,129.060822
64,starxo,123.308319
103,2ge,122.418465
